# Thesis Extraction for Manual Review

This notebook:
1. Loads PDF files from `data/latest_thesis_decks`
2. Uses GPT-4 to extract all thesis statements
3. Numbers each thesis uniquely
4. Generates 2-3 sentence descriptions
5. Identifies which theses support other theses
6. Exports to CSV for easy human review

**Output:** A clean CSV file with all theses ready for manual review and refinement.


In [ ]:
import os
import json
import re  # For JSON extraction
import time  # For waiting on assistant runs
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any
import fitz  # PyMuPDF
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
env_path = Path("../.env").resolve()
load_dotenv(env_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

DATA_FOLDER = Path("../data/latest_thesis_decks")
OUTPUT_FOLDER = Path("../data/analysis")
OUTPUT_FOLDER.mkdir(exist_ok=True, parents=True)

print("✓ Imports and configuration loaded")
print(f"  Data folder: {DATA_FOLDER.absolute()}")
print(f"  Output folder: {OUTPUT_FOLDER.absolute()}")
print(f"  OpenAI client: {'✓' if openai_client else '✗'}")


✓ Imports and configuration loaded
  Data folder: C:\Users\ChetKumar\Projects\thesis-bot\notebooks\..\data\latest_thesis_decks
  Output folder: C:\Users\ChetKumar\Projects\thesis-bot\notebooks\..\data\analysis
  OpenAI client: ✓


In [ ]:
# Step 1: Prepare PDF files for upload
# (No text extraction needed - we'll upload PDFs directly to OpenAI)

pdf_files = list(DATA_FOLDER.glob("*.pdf"))

if not pdf_files:
    print(f"⚠ No PDF files found in {DATA_FOLDER}")
    pdf_files = []

print(f"✓ Found {len(pdf_files)} PDF file(s) ready for upload")
for pdf_file in pdf_files:
    print(f"  - {pdf_file.name}")


Extracting text from: AI 20250923.pdf
  ✓ Extracted 45,833 characters
Extracting text from: Argonautic BioTech Presentation 20250412.pdf
  ✓ Extracted 28,462 characters
Extracting text from: Argonautic Consolidated Thesis Presentation 20250818.pdf
  ✓ Extracted 47,895 characters
Extracting text from: Argonautic Construction Technology 20250412.pdf
  ✓ Extracted 35,062 characters

✓ Loaded 4 PDF file(s)


In [ ]:
# Step 2: Extract theses using OpenAI Assistants API (handles PDFs directly)

def extract_theses_from_pdf(pdf_path: Path, filename: str) -> Dict[str, Any]:
    """
    Upload PDF directly to OpenAI and extract theses using Assistants API.
    This processes the entire PDF without truncation.
    """
    if not openai_client:
        print("⚠ OpenAI client not configured.")
        return {"theses": [], "thesis_supports": []}
    
    try:
        # Step 1: Upload the PDF file
        print(f"  Uploading {filename} to OpenAI...")
        with open(pdf_path, "rb") as file:
            uploaded_file = openai_client.files.create(
                file=file,
                purpose="assistants"
            )
        print(f"  ✓ File uploaded (ID: {uploaded_file.id})")
        
        # Step 2: Create an assistant with instructions
        assistant = openai_client.beta.assistants.create(
            name="Thesis Extractor",
            instructions="""You are an expert at analyzing documents and extracting thesis statements. 

Extract ALL thesis statements/arguments from the document. A thesis is a claim, argument, or proposition that the document is making.

For each thesis, provide:
1. The exact thesis statement (the claim being made)
2. A brief 2-3 sentence description that accurately describes what the thesis is about

Also identify relationships: if one thesis supports or provides evidence for another thesis, note that.

IMPORTANT:
- Extract ALL theses, even if they seem similar (we will deduplicate later)
- Be thorough - don't miss any arguments or claims
- Each thesis should be a distinct statement
- If theses are redundant or very similar, still list them (we'll handle deduplication)

Return a JSON object with this structure:
{
    "theses": [
        {
            "thesis": "The exact thesis statement",
            "description": "2-3 sentences accurately describing what this thesis is about"
        },
        ...
    ],
    "thesis_supports": [
        {
            "source_thesis": "The supporting thesis statement",
            "target_thesis": "The thesis statement it supports"
        },
        ...
    ]
}

Return ONLY valid JSON, no other text.""",
            model="gpt-4-turbo-preview",  # or "gpt-4o" for faster/cheaper
            tools=[{"type": "code_interpreter"}],  # Helps with document analysis
        )
        
        # Step 3: Create a thread and add the file
        thread = openai_client.beta.threads.create()
        
        message = openai_client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content="Extract all thesis statements from this document. Return the JSON structure as specified in your instructions.",
            file_ids=[uploaded_file.id]
        )
        
        # Step 4: Run the assistant
        print(f"  Analyzing document (this may take a minute)...")
        run = openai_client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id
        )
        
        # Step 5: Wait for completion
        max_wait = 300  # 5 minutes max
        wait_time = 0
        while run.status in ["queued", "in_progress"]:
            time.sleep(2)
            wait_time += 2
            if wait_time > max_wait:
                print(f"  ⚠ Timeout waiting for analysis")
                break
            run = openai_client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
            if wait_time % 10 == 0:
                print(f"    Still processing... ({wait_time}s)")
        
        if run.status == "completed":
            # Step 6: Get the response
            messages = openai_client.beta.threads.messages.list(
                thread_id=thread.id
            )
            
            # Extract JSON from the response
            result_text = messages.data[0].content[0].text.value
            
            # Try to extract JSON
            if "```json" in result_text:
                result_text = result_text.split("```json")[1].split("```")[0]
            elif "```" in result_text:
                result_text = result_text.split("```")[1].split("```")[0]
            
            # Find JSON object in the text
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)
            if json_match:
                result_text = json_match.group(0)
            
            result = json.loads(result_text.strip())
            result["filename"] = filename
            
            # Cleanup: delete the file and assistant
            try:
                openai_client.files.delete(uploaded_file.id)
                openai_client.beta.assistants.delete(assistant.id)
            except:
                pass  # Ignore cleanup errors
            
            return result
        else:
            print(f"  ✗ Run failed with status: {run.status}")
            if run.status == "failed":
                print(f"    Error: {run.last_error}")
            # Cleanup on failure
            try:
                openai_client.files.delete(uploaded_file.id)
                openai_client.beta.assistants.delete(assistant.id)
            except:
                pass
            return {"theses": [], "thesis_supports": [], "filename": filename}
            
    except Exception as e:
        print(f"Error analyzing {filename}: {e}")
        import traceback
        traceback.print_exc()
        return {"theses": [], "thesis_supports": [], "filename": filename}

# Extract theses from all PDFs using direct file upload
all_extractions = {}YY
for pdf_file in pdf_files:
    print(f"\n{'='*60}")
    print(f"Extracting theses from: {pdf_file.name}")
    print(f"{'='*60}")
    extraction = extract_theses_from_pdf(pdf_file, pdf_file.name)
    all_extractions[pdf_file.name] = extraction
    print(f"  ✓ Found {len(extraction.get('theses', []))} theses, {len(extraction.get('thesis_supports', []))} support relationships")

print(f"\n✓ Extraction complete for {len(all_extractions)} document(s)")



Extracting theses from: AI 20250923.pdf
  ✓ Found 11 theses, 0 support relationships

Extracting theses from: Argonautic BioTech Presentation 20250412.pdf
  ✓ Found 9 theses, 0 support relationships

Extracting theses from: Argonautic Consolidated Thesis Presentation 20250818.pdf
  ✓ Found 18 theses, 0 support relationships

Extracting theses from: Argonautic Construction Technology 20250412.pdf
  ✓ Found 7 theses, 0 support relationships

✓ Extraction complete for 4 document(s)


In [ ]:
# Step 3: Deduplicate and number theses

def deduplicate_theses(all_extractions: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    Combine theses from all documents, deduplicate similar ones, and assign unique numbers.
    """
    all_theses = []
    thesis_to_number = {}
    thesis_number = 1
    
    # First pass: collect all theses with their source files
    for filename, extraction in all_extractions.items():
        for thesis_data in extraction.get("theses", []):
            thesis = thesis_data.get("thesis", "").strip()
            description = thesis_data.get("description", "").strip()
            
            if thesis:  # Only add non-empty theses
                all_theses.append({
                    "thesis": thesis,
                    "description": description,
                    "source_file": filename
                })
    
    # Second pass: deduplicate similar theses
    # Group theses by similarity (simple approach: exact match or very similar)
    unique_theses = []
    seen_theses = set()
    
    for thesis_data in all_theses:
        thesis = thesis_data["thesis"]
        thesis_lower = thesis.lower().strip()
        
        # Check if we've seen a very similar thesis
        is_duplicate = False
        for seen in seen_theses:
            # Simple similarity check: if one is contained in the other or very similar
            if (thesis_lower in seen.lower() or seen.lower() in thesis_lower) and abs(len(thesis_lower) - len(seen)) < 20:
                is_duplicate = True
                break
        
        if not is_duplicate:
            seen_theses.add(thesis)
            unique_theses.append(thesis_data)
            thesis_to_number[thesis] = thesis_number
            thesis_number += 1
    
    # Build thesis supports relationships with numbers
    thesis_supports = []
    for filename, extraction in all_extractions.items():
        for rel in extraction.get("thesis_supports", []):
            source = rel.get("source_thesis", "").strip()
            target = rel.get("target_thesis", "").strip()
            
            # Find the unique thesis numbers
            source_num = None
            target_num = None
            
            for thesis, num in thesis_to_number.items():
                if source and (source in thesis or thesis in source):
                    source_num = num
                if target and (target in thesis or thesis in target):
                    target_num = num
            
            if source_num and target_num:
                thesis_supports.append({
                    "source_thesis_number": source_num,
                    "target_thesis_number": target_num,
                    "source_thesis": source,
                    "target_thesis": target
                })
    
    return {
        "theses": unique_theses,
        "thesis_to_number": thesis_to_number,
        "thesis_supports": thesis_supports
    }

# Deduplicate and number
print("Deduplicating theses...")
deduplicated = deduplicate_theses(all_extractions)
print(f"✓ Found {len(deduplicated['theses'])} unique theses (from {sum(len(e.get('theses', [])) for e in all_extractions.values())} total)")
print(f"✓ Found {len(deduplicated['thesis_supports'])} support relationships")


In [ ]:
# Step 4: Create review-friendly CSV

def create_review_csv(deduplicated: Dict[str, Any], output_file: Path):
    """
    Create a CSV file optimized for human review.
    """
    thesis_to_number = deduplicated["thesis_to_number"]
    thesis_supports = deduplicated["thesis_supports"]
    
    # Build a map of which theses each thesis supports
    supports_map = {}
    for rel in thesis_supports:
        source_num = rel["source_thesis_number"]
        target_num = rel["target_thesis_number"]
        
        if source_num not in supports_map:
            supports_map[source_num] = []
        supports_map[source_num].append(target_num)
    
    # Create rows for CSV
    rows = []
    for thesis_data in deduplicated["theses"]:
        thesis = thesis_data["thesis"]
        thesis_num = thesis_to_number[thesis]
        
        # Get list of thesis numbers this thesis supports
        supports_list = sorted(supports_map.get(thesis_num, []))
        supports_str = ", ".join(map(str, supports_list)) if supports_list else ""
        
        rows.append({
            "Thesis Number": thesis_num,
            "Thesis Statement": thesis,
            "Description": thesis_data["description"],
            "Supports Thesis Numbers": supports_str,
            "Source File": thesis_data["source_file"]
        })
    
    # Sort by thesis number
    rows.sort(key=lambda x: x["Thesis Number"])
    
    # Create DataFrame and save
    df = pd.DataFrame(rows)
    df.to_csv(output_file, index=False)
    
    return df

# Create the review CSV
output_file = OUTPUT_FOLDER / "theses_for_review.csv"
df = create_review_csv(deduplicated, output_file)

print(f"\n✓ Created review CSV: {output_file}")
print(f"\nCSV Summary:")
print(f"  Total theses: {len(df)}")
print(f"  Theses with support relationships: {len(df[df['Supports Thesis Numbers'] != ''])}")
print(f"\nFirst few rows:")
print(df.head(10).to_string())
print(f"\n📝 Next steps:")
print(f"  1. Open {output_file} in Excel or a text editor")
print(f"  2. Review each thesis statement and description")
print(f"  3. Ensure descriptions are 2-3 sentences and accurate")
print(f"  4. Verify and update 'Supports Thesis Numbers' column")
print(f"  5. Save as 'theses_reviewed.csv' when done")
